In [1]:
import cv2
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import random

In [67]:
DEFAULT_BACKGROUND_COLOUR = 250
CHANNELS = 3
VAR = 9

class PaperGeneration:
    def __init__(self, width, height, colour = DEFAULT_BACKGROUND_COLOUR, channels = CHANNELS, var = VAR):
        self.colour = colour
        self.var = var
        self.channels = channels
        pure_paper = self.create_paper(width, height)
        textured_paper = self.add_texture(pure_paper)
        self.noised_textured_paper = self.add_noise(textured_paper)

    def create_paper(self, height, width):
        image = np.full((height, width, self.channels), self.colour, dtype=np.uint8)
        return image

    def add_texture(self, image):
        texture = cv2.imread('/home/arsen/Textures/texture' + str(random.randint(1,3)) + '.png')
        texture = cv2.resize(texture, (image.shape[1], image.shape[0]))
        weight1 = random.uniform(0.3,0.6)
        weight2 = 1 - weight1
        textured_image = cv2.addWeighted(image, weight1, texture, weight2, 0, dtype=cv2.CV_32F)
        return textured_image

    def add_noise(self, image, mean=0, ratio=1):
        width, height, channels = image.shape
        new_height = int(height / ratio)
        new_width = int(width / ratio)
        mask = np.random.normal(mean, self.var**0.5, (new_width, new_height, self.channels))
        if ratio > 1:
            mask = cv2.resize(mask, dsize=(width, height), interpolation=cv2.INTER_LINEAR)
        image = image + mask.reshape((width, height, CHANNELS))
        return np.clip(image, 0, 255)

    def get_image(self):
        return self.noised_textured_paper



In [90]:
paper_gen = PaperGeneration(width=4096, height=4096)
paper = paper_gen.get_image()

In [133]:
class ImageAugmentation:
    def __init__(self, paper, channel_shift, translation_shift, rotation_angle, blur_max_kernel):
        self.paper = paper
        self.value = channel_shift
        self.shift = translation_shift
        self.rotation_angle = rotation_angle
        self.kernel = blur_max_kernel
        background_path = '/home/arsen/Backgrounds/background' + str(np.random.randint(1, 4)) + '.png'
        self.background_image = cv2.imread(background_path)
        
    def scale(self, image, scale):
        dim = (int(image.shape[1] * scale), int(image.shape[0] * scale))
        image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        return image
    
    def channel_shift(self, imаgе):
        value = int(np.random.uniform(-self.value, self.value))
        imаgе = np.clip(imаgе + value, 0, 255).astype(np.uint8)
        return imаgе
    
    def translation(self, image):
        h, w = image.shape[:2]
        w_shift = int(np.random.uniform(-self.shift, self.shift))
        h_shift = int(np.random.uniform(-self.shift, self.shift))
        M = np.array([[1, 0, w_shift], [0, 1, h_shift]], dtype=np.float32)
        image = cv2.warpAffine(image, M, (w, h), borderValue = (1, 1, 1))
        return image
    
    def rotation(self, image):
        angle = int(np.random.uniform(-self.rotation_angle, self.rotation_angle))
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
        image = cv2.warpAffine(image, M, (w, h), borderValue=(1, 1, 1))
        return image
    
    def blur(self, image):
        kernel = np.random.randint(1, self.kernel)
        return cv2.blur(image, (kernel, kernel))
    
    def put_image_on_paper(self, image, paper):
        mask = np.zeros_like(image)
        mask[:, :, 0] = mask[:, :, 1] = mask[:, :, 2] = (image[:, :, 0] == 255) * (image[:, :, 1] == 255) * (image[:, :, 2] == 255)
        mask = mask.astype(bool)
        return paper[:image.shape[0], :image.shape[1]] * mask + image * ~mask
    
    def put_image_on_background(self, image, background):
        mask = np.zeros_like(image)
        mask[:, :, 0] = mask[:, :, 1] = mask[:, :, 2] = (image[:, :, 0] == 1) * (image[:, :, 1] == 1) * (image[:, :, 2] == 1)
        mask = mask.astype(bool)
        return background * mask + image * ~mask
    
    def print_defect(self, imаgе):
        shifted_rows = np.random.binomial(1, 0.01, imаgе.shape[0])
        for i, shifted in enumerate(shifted_rows):
            if shifted:
                shift = random.randint(-25, 25)
                imаgе[i] = np.roll(imаgе[i], shift, axis=0)
        return imаgе
    
    def augmentate(self, image):
        new_background = self.scale(self.background_image, 2)
        new_image = self.scale(image, np.random.uniform(0.3, 0.7))
        image_on_paper = self.put_image_on_paper(new_image, paper)
        new_image = self.print_defect(image_on_paper)
        trick = np.ones_like(new_background)
        trick[500:500 + new_image.shape[0], 500:500 + new_image.shape[1]] = new_image
        new_image = self.translation(self.rotation(trick))
        image_on_background = self.put_image_on_background(new_image,new_background)
        return self.blur(image_on_background)

In [136]:
os.mkdir('/home/arsen/Output_dir')
for file in tqdm(os.listdir('/home/arsen/Images')):
    image = cv2.imread('/home/arsen/Images/' + file)
    for num in range(10):
        aug = ImageAugmentation(paper,60,500,45,10)
        cv2.imwrite('/home/arsen/Output_dir/' + file[:-4]+ '_' + str(num) + '.png', aug.augmentate(image))

100% 10/10 [01:05<00:00,  6.55s/it]
